In [1]:
import numpy as np
import random
from higherorder.structures.structures import Grid, Graph, Structure
from higherorder.dynamics.model import Model
import higherorder.analysis.ca as simulations
from higherorder.dynamics.rules import *
from higherorder.dynamics.impacts import *
from higherorder.utils import *

Load randomly initialized grids

In [2]:
grids_dicts1 = load_init_grid_dicts(".\\data\\oneblob_grids.json", listify=True)
grids_dicts2 = load_init_grid_dicts(".\\data\\twoblob_grids.json", listify=True)
grids_dicts3 = load_init_grid_dicts(".\\data\\two_from_oneblob.json", listify=True)

Cast back to grid

In [3]:
height = 20
width = 20

grids1 = []
for i in range(len(grids_dicts1)):
    grids1.append(Grid(grids_dicts1[i], width=width, height=height))

grids3 = []
for i in range(len(grids_dicts3)):
    grids3.append(Grid(grids_dicts3[i], width=width, height=height))

This takes considerably longer to load than the other two sources (19000 grids vs 2x2400 grids)

In [ ]:
grids2 = []
for i in range(len(grids_dicts2)):
    grids2.append(Grid(grids_dicts2[i], width=width, height=height))

Run Game of Life simulation till repeating structure - better would be till repeating blobs (but also consider e.g. non stop till blobs "have a chance to meet" e.g. go in the same direction)

In [4]:
states1 = []
impact1 = []
for i in range(len(grids1)):
    sim = Model(grids1[i], dynamics_func=game_of_life,)
    sim.simulate_till_periodicity(store_impact=True, impact_function=game_of_life_impact)
    states1.append(sim.structure.get_entities_states())
    impact1.append(sim.impact)

In [ ]:
states3 = []
impact3 = []
for i in range(len(grids3)):
    sim = Model(grids3[i], dynamics_func=game_of_life,)
    sim.simulate_till_periodicity(store_impact=True, impact_function=game_of_life_impact)
    states3.append(sim.structure.get_entities_states())
    impact3.append(sim.impact)

In [ ]:
states2 = []
impact2 = []
for i in range(len(grids2)):
    sim = Model(grids2[i], dynamics_func=game_of_life,)
    sim.simulate_till_periodicity(store_impact=True, impact_function=game_of_life_impact)
    states2.append(sim.structure.get_entities_states())
    impact2.append(sim.impact)

Across all examples: pick 2 random neighboring nodes (or aggregate across such) which:
- did not impact each other
- did impact each other
- had turn off impact e.g.?
- had 0,0 all the time
- had 0,1 or 0,0 only
- had 0,1 and 1,0 too
- had 1,1

Mutual information is enough?
For not pairs, we would use HOI

Watch out for time delay. Need to consider: spatially moving entropy?

## Analysis of the 8 cells around a target cell 

All possible combinations

In [ ]:
def dec_to_bin(x, binary_length=9):
    return np.array([int(b) for b in format(x, f'0{binary_length}b')])

def bin_to_dec(bin, binary_length=9):
    return sum([pow(2,binary_length-j-1)*i for j,i in enumerate(bin)])

nodes = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
states_all = []
impacts_all = []
categories = {}

for i in range(512):
    binary_vals = dec_to_bin(i)
    initial_arr = np.zeros((3, 3))
    for j in range(len(nodes)):
        initial_arr[nodes[j]] = binary_vals[j]
    grid = Grid(initial_arr, width=3, height=3)
    sim = Model(grid, dynamics_func=game_of_life,)
    sim.simulation(steps=1, store_impact=True, impact_function=game_of_life_impact)
    states = sim.structure.get_time_slice(key_name="t_1", only_nonzero=False)
    states_all.append(states)
    impacts_all.append(sim.impact)
    
    around = (binary_vals).copy()
    around[4] = 0
    s = np.sum(around)
    if s not in categories.keys():
        categories[s] = []
    categories[s].append((binary_vals,states[(1,1)], sim.impact)) #TODO

In [91]:
time_arrays = {}
y_values = {}
for n in range(9):
    instances = len(categories[n])
    time_array = np.zeros((instances, 9))
    ys = np.zeros(instances)
    for j,instance in enumerate(categories[n]):
        for k in range(9):
            time_array[j][k] = instance[0][k]
        ys[j] = instance[1]
    time_arrays[n] = time_array
    y_values[n] = ys

1) Synergy and redundancy separately

In [177]:
from hoi.metrics import DTC, TC, RedundancyMMI, SynergyMMI

Node may have impact on itself - implement

2) Finish HOI, make plot of points, e.g. comparing

In [ ]:
from hoi.metrics import Oinfo, GradientOinfo
from hoi.simulation import simulate_hoi_gauss
import matplotlib.pyplot as plt

def compute_hoi_beh(x):
    """This function computes the HOI using the Oinfo."""
    model = Oinfo(x)
    hoi = model.fit(method="binning", minsize=3, maxsize=3)
    return hoi.squeeze()

def compute_hoi_enc(x, y):
    """This function computes the HOI using the Oinfo."""
    model = GradientOinfo(x, y)
    hoi = model.fit(method="binning", minsize=3, maxsize=3)
    return hoi.squeeze()

def compute_hoi_enc_nonflat(x, y):
    """This function computes the HOI using the Oinfo."""
    model = GradientOinfo(x, y)
    hoi = model.fit(method="binning", minsize=3, maxsize=3)
    return hoi


birth_array = np.array([[1, 1, 1], 
                        [1, 1, 0], 
                        [1, 0, 1],
                        [0, 1, 1]])
#birth_array = np.concatenate([birth_array, birth_array, birth_array, birth_array, birth_array], axis=1)
birth_target = np.array([1, 0, 0, 0])

birth_hoi = compute_hoi_enc_nonflat(birth_array, birth_target)[0][0]
print(birth_hoi)

kill_array = np.array([[1, 1, 1, 1],
                        [1, 1, 1, 0], 
                        [1, 1, 0, 1],
                        [1, 0, 1, 1]])
kill_target = np.array([0, 1, 1, 1])
kill_hoi = max(compute_hoi_enc_nonflat(kill_array, kill_target))[0]
print(kill_hoi)

redundancy_kill_array = np.array([[1, 1, 1, 1, 1],
                                  [1, 1, 1, 1, 0],
                                  [1, 1, 1, 0, 1], 
                                  [1, 1, 0, 1, 1],
                                  [1, 0, 1, 1, 1],
                                  [0, 1, 1, 1, 0], # for comparison
                                  ])
redundancy_kill_target = np.array([0, 0, 0, 0, 0, 1])
redundancy_kill_hoi = (compute_hoi_enc_nonflat(redundancy_kill_array, redundancy_kill_target))
#nonactive_redundancy_live

In [193]:
redundancy_kill_hoi

array([[ 0.10917044],
       [ 0.10917044],
       [ 0.316689  ],
       [ 0.10917044],
       [ 0.316689  ],
       [ 0.316689  ],
       [-0.05423784],
       [ 0.10917044],
       [ 0.10917044],
       [ 0.10917044]], dtype=float32)

TODO check separately measures of redundancy, and synergy

TC: total correlation, associated with redundancy; high -> high strength of collective constraints

DTC: low -> high synergy

RSI: redundancy synergy index

RedundancyMMI, SynergyMMI

Try DTC, TC, and RedundancyMMI and SynergyMMI


Combine two "neighboring" categories for better analysis

3)  Group theory: 3.1 check gliders in and out impact, and other self-ref machines
4) take many generated self-ref

Connect it with impact? e.g. impact can cover synergy, opposite can cover redundancy possibly? Only select impacts going to the target cell

## Other analysis

In [ ]:
for i in range(height):
    for j in range(width):
        cell = (i, j)
        pass

Variables array creation - watch out to store the same nodes in the same columns across arrays

Then use HOI on random neighboring nodes, random nodes at distance d, then do the same with only "inside blob only" nodes (how?)

Impact- into simulation functions as option, for now only GoL impact implementation.